In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir drive
!google-drive-ocamlfuse drive

In [2]:
import os
from keras.preprocessing.text import text_to_word_sequence
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Sequential
import numpy as np

Using TensorFlow backend.


In [3]:
#1. Processing data
#load data
file_list = []
for (dirpath, dirname, filename) in os.walk('drive/NLP/data'):
    for f in filename:
        file_list.append(dirpath+'/'+f)

20


In [0]:
#tokenizer document
def clean_document(doc):
    return text_to_word_sequence(doc)

In [0]:
#doc->sentence->sequence = sentences
INPUT_LENGTH = 50
sequences = []
for f in file_list:
    f1 = open(f, encoding='utf8')
    doc = f1.read()
    tokens = clean_document(doc)

    for i in range(INPUT_LENGTH + 1, len(tokens)):
        seq = tokens[i-INPUT_LENGTH-1:i]
        line = ' '.join(seq)
        sequences.append(line)

In [0]:
#remove special character, create dict vocab
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~ _')
tokenizer.fit_on_texts(sequences)
vocab_size = len(tokenizer.word_index) + 1

In [0]:
#word to digit
sequences_digit = tokenizer.texts_to_sequences(sequences)

sequences_digit = np.array(sequences_digit)
X, y = sequences_digit[:,:-1], sequences_digit[:,-1]

#one hot vector
y = keras.utils.to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [0]:
#2. Model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=50))
model.add(BatchNormalization())
model.add(LSTM(512, return_sequences=True))
model.add(LSTM(512))
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(vocab_size, activation='softmax'))
model.add(Dropout(0.2))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 50)            279200    
_________________________________________________________________
batch_normalization_3 (Batch (None, 50, 50)            200       
_________________________________________________________________
lstm_3 (LSTM)                (None, 50, 512)           1153024   
_________________________________________________________________
lstm_4 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_3 (Dense)              (None, 100)               51300     
_________________________________________________________________
batch_normalization_4 (Batch (None, 100)               400       
_________________________________________________________________
dense_4 (Dense)              (None, 5584)              563984    
__________

In [0]:
filepath = "drive/NLP/english_language_model.hdf5"
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint(filepath, monitor='acc', mode='max', verbose=1, save_best_only=True)
# fit model
model.fit(X,y, batch_size=512, epochs=10,callbacks=[es,mc])


Epoch 1/10
26756/26756 [==============================] - 609s 23ms/step - loss: 9.7245 - acc: 0.0513

Epoch 00001: acc improved from -inf to 0.05132, saving model to drive/NLP/english_language_model.hdf5
Epoch 2/10
26756/26756 [==============================] - 605s 23ms/step - loss: 9.0785 - acc: 0.0955

Epoch 00002: acc improved from 0.05132 to 0.09549, saving model to drive/NLP/english_language_model.hdf5
Epoch 3/10
26756/26756 [==============================] - 599s 22ms/step - loss: 8.3483 - acc: 0.1286

Epoch 00003: acc improved from 0.09549 to 0.12861, saving model to drive/NLP/english_language_model.hdf5
Epoch 4/10
26756/26756 [==============================] - 595s 22ms/step - loss: 7.7635 - acc: 0.1590

Epoch 00004: acc improved from 0.12861 to 0.15896, saving model to drive/NLP/english_language_model.hdf5
Epoch 5/10
26756/26756 [==============================] - 596s 22ms/step - loss: 7.3461 - acc: 0.1832

Epoch 00005: acc improved from 0.15896 to 0.18325, saving model to d

In [0]:
#20
model.fit(X,y, batch_size=512, epochs=10,callbacks=[es,mc])

Epoch 1/10
26756/26756 [==============================] - 619s 23ms/step - loss: 5.3439 - acc: 0.4162

Epoch 00001: acc improved from 0.35820 to 0.41621, saving model to drive/NLP/english_language_model.hdf5
Epoch 2/10
26756/26756 [==============================] - 620s 23ms/step - loss: 4.9517 - acc: 0.4791

Epoch 00002: acc improved from 0.41621 to 0.47914, saving model to drive/NLP/english_language_model.hdf5
Epoch 3/10
26756/26756 [==============================] - 614s 23ms/step - loss: 4.5627 - acc: 0.5488

Epoch 00003: acc improved from 0.47914 to 0.54877, saving model to drive/NLP/english_language_model.hdf5
Epoch 4/10
26756/26756 [==============================] - 610s 23ms/step - loss: 4.2627 - acc: 0.6124

Epoch 00004: acc improved from 0.54877 to 0.61242, saving model to drive/NLP/english_language_model.hdf5
Epoch 5/10
26756/26756 [==============================] - 606s 23ms/step - loss: 4.0121 - acc: 0.6723

Epoch 00005: acc improved from 0.61242 to 0.67226, saving model t

In [0]:
#save tokenier and sequence digits
import pickle
with open('drive/NLP/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)
with open('drive/NLP/sequences_digit.pkl', 'wb') as f:
    pickle.dump(sequences_digit, f)

In [0]:
#3.Application
#load model to generate text
from keras.models import load_model

with open('drive/NLP/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)
    
with open('drive/NLP/sequences_digit.pkl', 'rb') as f:
    sequences_digit = pickle.load(f)

model = load_model('drive/NLP/english_language_model.hdf5')




In [0]:
doc = "that is not essential"

In [0]:
def generate_text(seed_text, next_words, max_sequence_len, model):
    for j in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen= 
                             max_sequence_len, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
  
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text


In [67]:
print(generate_text(doc,15,50,model))

that is not essential fixer the development of this approaches not contain the development of communication which technologies can


In [0]:
#Comment: hình như mô hình học thuộc lòng :)